# Attention Mechanism
디코더에서 출력 단어를 예측하는 매 시점(time step)마다, 인코더에서의 전체 입력 문장을 다시 한 번 참고한다.


 ![image.png](attachment:ca5142bc-f2ae-4072-9724-8a2c2ba5366e.png)

### Attention 함수
>- Q = Query : t 시점의 디코더 셀에서의 은닉 상태
>- K = Keys : 모든 시점의 인코더 셀의 은닉 상태
>- V = Values : 모든 시점의 인코더 셀의 은닉 상태

### Dot-Product Attention


1. Attention score((디코더의 현재 상태와 인코더의 은닉 상태들에 대해 dot product)를 구한다. => Attention score 모음값![image.png](attachment:2c4d68df-00c8-4f85-b926-655ad8342c90.png)를 구한다. => 이에 대해 softmax를 적용하여 어텐션 분포(어텐션 가중치에 대한)를 구한다. => 어텐션 가중치와 인코더의 은닉 상태를 가중합한다. => 디코더의 현재 상태와 결합(concatenation)하여 ![image.png](attachment:24ed93fb-75e2-40cd-b1cb-a4fa3e3be082.png)를 구한다. => vt를 가중치 행렬과 곱한 후 tanh함수를 적용하여 ![image.png](attachment:06ba397d-823c-4c83-9698-97d98fbaaec2.png)를 얻는다. 출력층의 입력으로 이를 사용한다.


![image.png](attachment:42257e74-b476-4ee0-bdf2-fb33c6e2c20f.png)

![image.png](attachment:f2d9c325-1dfa-41d9-ac6f-ade4fb204146.png)

![image.png](attachment:784ad924-d27b-428e-a752-8fbf8f810e51.png)

![image.png](attachment:158a5617-7cfc-4124-9aa3-cda0c7409d4c.png)

![image.png](attachment:3d7adb0e-ed76-43af-87bc-cd33eb5d6ca8.png)







In [2]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [3]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocab_size)


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
print('리뷰의 최대 길이 : {}'.format(max(len(l) for l in X_train)))
print('리뷰의 평균 길이 : {}'.format(sum(map(len, X_train))/len(X_train)))


리뷰의 최대 길이 : 2494
리뷰의 평균 길이 : 238.71364


In [5]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)


In [6]:
import tensorflow as tf

class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    # values에 적용되는 가중치
    self.W2 = Dense(units)
    # query에 적용되는 가중치
    self.V = Dense(1)
    # 스코어를 계산하는 가중치

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights


In [7]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os


sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero = True)(sequence_input)


lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences = True))(embedded_sequences)

lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)


print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs = 3, batch_size = 256, validation_data=(X_test, y_test), verbose=1)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:915: UserWarning: Layer 'bahdanau_attention' (of type BahdanauAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


98/98 ━━━━━━━━━━━━━━━━━━━━ 1085s 11s/step - accuracy: 0.6280 - loss: 0.6174 - val_accuracy: 0.8674 - val_loss: 0.3142
Epoch 2/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 1122s 11s/step - accuracy: 0.9002 - loss: 0.2666 - val_accuracy: 0.8848 - val_loss: 0.2808
Epoch 3/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 1081s 11s/step - accuracy: 0.9304 - loss: 0.2014 - val_accuracy: 0.8872 - val_loss: 0.2752


In [8]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))


782/782 ━━━━━━━━━━━━━━━━━━━━ 299s 382ms/step - accuracy: 0.8868 - loss: 0.2770

 테스트 정확도: 0.8872
